In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
property_ids_url = "https://subastas.boe.es/reg/subastas_ava.php?accion=Mas&id_busqueda=_ZURMUzR4WmN2Uk1VS1dkeEo1ZmFId0ppTUxjVVVQclNub3BnckVtQzdxeDJEY2Z6V2dOWVJRT0pqTlFlS01YVUUzZFdISmJpeTBEQVN1TVpnSGpodzNTYkhxNWo3ejY4eGNQVGZ1dHhCQ1hXM0lFZG1tMEhETGtBSE13ZGtiREZOSUh1d3RXMWFIZkVqNCtGbUhtWm1nd0Q4QWNYYkJrZHpqdzVDVFNOY094MFZkdkF5U2kvSXQ4Z0N4STBZakNwV0hnUnByWG5mMXRTZStidnc3YlByaEMvZXFSVkdDZXlJazF0eDlTK0hETTdMT0h4S3p6NEJJd21hdHEreXpZZVhyclFsdFI4RTRNVlBJbGlGbThicTNpT1NTWGVaNVRvRlpnMGN3Ky8xeHJHQmJDaGxBR0g3Um12V2FsbEJaZnM,-0-50"

# Fetch the HTML content
response = requests.get(property_ids_url)
html_content = response.text
# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Extract property IDs from the <a> elements with class 'resultado-busqueda-link-otro'
properties_list = [a['title'] for a in soup.find_all('a', class_='resultado-busqueda-link-otro')]

# Initialize an empty DataFrame to store the property info
df = pd.DataFrame()

# Loop through property IDs and extract info
for property_id_full in properties_list:
    # Extract the relevant part of the property ID
    property_id = property_id_full.split(" ")[1]
    
    # Construct property URL
    property_url = f"https://subastas.boe.es/detalleSubasta.php?idSub={property_id}"
    
    # Fetch the property page
    response = requests.get(property_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Check if the property page has the expected structure
    table_div = soup.find("div", id="idBloqueDatos1")
    if table_div:
        # Extract info from the table
        table = table_div.find("table")
        rows = table.find_all("tr")
        
        property_data = {}
        for row in rows:
            cols = row.find_all(["th", "td"])  # Including th (headers) and td (values)
            header = cols[0].get_text(strip=True)
            value = cols[1].get_text(strip=True) if len(cols) > 1 else ""  # Handle cases where there's no value cell
            property_data[header] = value
        
        # Append the property info to the DataFrame
        df = pd.concat([df, pd.DataFrame(property_data, index=[0])], ignore_index=True)

# Create a list to store dictionaries of property data
data_list = []

# Iterate through each property
for property_info in properties_list:
    if property_info.startswith('Subasta SUB-'):
        property_id = property_info.split(' ')[-1]
        
        url = f'https://subastas.boe.es/reg/detalleSubasta.php?idSub={property_id}&ver=3&idBus=_aGZaODZVK09aRlgzcFpxSzQ4SWlXa3ZOSDdnaFNaMkg1OWV0U2pHWjFwUEc2aHo5TVl3QjNtUnJUQ1FGS2ZoYmdoYS94cncxU1ZhYjNYMXFndzdyaVNvUlZpSUJHdm83eEVla3UyMElJYjJFcHpGdFB4N0RWQ20xRThPNFFrUStieGtWOWhaTlJuNGtqUXlyV3V3dFdYbWFPd3BGMnh3dlZaK2tXaElFQjVvZmhiUkdFMTJJYTYrUnRsMkU0enNHektmOUZRdnpYQmZHcERYRlY2bUdDVHE1UFZaLzJRS1BDMFlPNVhCb2R1SEYrOGljOW85djdqdE1zeWZySnNZQS8wTkhSS1lvWkxVMGJ5enJkWUhLdEhmYy9JY3g0YlF3YnpZdWo1aVhlOGs9--50&idLote=&numPagBus='

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        div_id_bloque_datos3 = soup.find('div', id='idBloqueDatos3')
        if div_id_bloque_datos3:
            data_divs = div_id_bloque_datos3.find_all('div', class_='bloque')
            for data_div in data_divs:
                property_data = {'Descripción': property_info, 'Property ID': property_id}
                
                property_table = data_div.find('table')
                if property_table:
                    rows = property_table.find_all('tr')
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        if th and td:
                            property_data[th.get_text(strip=True)] = td.get_text(strip=True)
                
                data_list.append(property_data)

# Create a DataFrame from the list of dictionaries
bienes_df = pd.DataFrame(data_list)
bienes_df.rename(columns={'Property ID':'Identificador'},inplace=True)
columns_to_drop = ['Título jurídico', 'Información adicional', 'Valor Subasta', 'Valor de tasación','Importe del depósito', 'Puja mínima', 'Tramos entre pujas','IDUFIR']

bienes_df = bienes_df.drop(columns=columns_to_drop)

main_df = df.merge(bienes_df, on='Identificador', how='inner')
main_df = main_df.dropna(subset=['Dirección', 'Código Postal'])
main_df['Localidad'] = main_df['Localidad'].fillna('Consuela')
main_df.fillna('No consta', inplace=True)
columns_to_convert = ['Dirección', 'Código Postal', 'Localidad', 'Provincia']
main_df[columns_to_convert] = main_df[columns_to_convert].apply(lambda x: x.str.upper())

# Drop the 'Forma adjudicación' column
df.drop('Forma adjudicación', axis=1, inplace=True)

# Convert 'Fecha de inicio' and 'Fecha de conclusión' to datetime
def parse_datetime(date_string):
    return pd.to_datetime(date_string.split('CET')[0].strip(), format='%d-%m-%Y %H:%M:%S')

df['Fecha de inicio'] = df['Fecha de inicio'].apply(parse_datetime)
df['Fecha de conclusión'] = df['Fecha de conclusión'].apply(parse_datetime)

df['Fecha de inicio'] = df['Fecha de inicio'].dt.date
df['Fecha de conclusión'] = df['Fecha de conclusión'].dt.date

# Convert numeric columns to float with correct formatting
numeric_columns = ['Cantidad reclamada', 'Valor subasta', 'Tasación', 'Importe del depósito']
for col in numeric_columns:
    df[col] = df[col].apply(lambda x: x.replace(' €', '').replace('.', '').replace(',', '.') if isinstance(x, str) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')

special_numeric_columns = ['Puja mínima', 'Tramos entre pujas']

for col in special_numeric_columns:
    df[col] = df[col].apply(lambda x: 0 if x == 'Sin tramos' else x)
    df[col] = df[col].apply(lambda x: x.replace(' €', '').replace('.', '').replace(',', '.') if isinstance(x, str) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
# Fill NaN values
columns_to_fill = ['Valor subasta', 'Puja mínima', 'Cantidad reclamada', 'Tasación','Tramos entre pujas','Importe del depósito']
df[columns_to_fill] = df[columns_to_fill].fillna(0)
df['Cuenta expediente'] = df['Cuenta expediente'].fillna('0123 4567 89 0987 65')

# Replace 'Sin lotes' with 0
df['Lotes'] = df['Lotes'].replace('Sin lotes', 0)

# Convert the 'Lotes' column to integer type
df['Lotes'] = df['Lotes'].astype(int)

# List of first words to filter out
words_to_filter = [
    'LG', 'TN', 'PEBRES', 'VIVIENDA', 'CP', 'MN', '30005', 'PD', 'AR', 'CN',
    'GREGORIO', 'NUEVA', 'POU', 'RAMON', 'PARAJE', 'PARTIDA', 'DISEMINADO',
    'SAN', 'PEREIJO.', 'SANTA', 'CAÑOCLAR', 'NO', 'CLOSA', 'GRAN', 'LA', 'SUERTE'
]

# Filter out rows with the specified first words in the address
filtered_df = main_df[~main_df['Dirección'].str.split().str[0].isin(words_to_filter)]

# Mapping of first words to standardized forms
standardized_mapping = {
    'CL': 'CALLE',
    'LUGAR': 'LUGAR',
    'CAMINO': 'CAMINO',
    'CALLE': 'CALLE',
    'AV.': 'AVENIDA',
    'CL.': 'CALLE',
    'C/': 'CALLE',
    'C/COSTERETA,': 'CALLE COSTERETA',
    'UR': 'URBANIZACION',
    'C.': 'CALLE',
    'CM': 'CAMINO',
    'CR': 'CALLE',
    'C/SANTIAGO': 'CALLE SANTIAGO',
    'PA': 'PASAJE',
    'CARRETERA': 'CARRETERA',
    'AV': 'AVENIDA',
    'PJ': 'PASAJE',
    'C/VIRGEN': 'CALLE VIRGEN',
    'CALLE:': 'CALLE',
    'PARQUE': 'PARQUE',
    'PZ': 'PLAZA',
    'POLIGONO': 'POLIGONO',
    'PG': 'POLIGONO',
    'POLÍGONO': 'POLIGONO',
    'PLAZA': 'PLAZA',
    'C/BLASCO': 'CALLE BLASCO',
    'PASAJE': 'PASAJE',
    'C/CHILE': 'CALLE CHILE',
    'C/TEIDE': 'CALLE TEIDE',
    'C/ALFONSO': 'CALLE ALFONSO',
    '"EDIFICIO': 'EDIFICIO',
    'CL/': 'CALLE',
    'AVENIDA': 'AVENIDA',
    'NUMERO': 'NUMERO',
    'URB.': 'URBANIZACION',
    'C/ANTONIO': 'CALLE ANTONIO',
    'URB': 'URBANIZACION',
    'RONDA': 'RONDA',
    'RAMBLA': 'RAMBLA',
    'URBANIZACION': 'URBANIZACION',
    'C/LUÍS': 'CALLE LUIS',
    'C/NICOLÁS': 'CALLE NICOLAS',
    'PG.': 'POLIGONO',
    'AVDA': 'AVENIDA',
    'TRAVESIA': 'TRAVESIA',
    'RAMAL': 'RAMAL',
    'CTRA': 'CARRETERA',
    'PASEO': 'PASEO',
    'SOLAR': 'SOLAR',
    'CAMI': 'CAMINO',
    'C/SANTO': 'CALLE SANTO',
    'AVD': 'AVENIDA',
    'PARCELA': 'PARCELA',
    'AVDA.': 'AVENIDA'
}

# Apply standardization to the 'Dirección' column
filtered_df['Dirección'] = filtered_df['Dirección'].apply(lambda address: ' '.join([standardized_mapping.get(address.split()[0], address.split()[0].upper())] + address.split()[1:]))

# Function to extract the address part until the first number
def extract_address(address):
    match = re.match(r'^(.*?\d+)\b', address)
    if match:
        return match.group(1)
    return address

# Create the "Dirección Mapa" column
filtered_df['Dirección Mapa'] = filtered_df['Dirección'].apply(extract_address)

In [2]:
# # URL to fetch property IDs
# # property_ids_url = "https://subastas.boe.es/reg/subastas_ava.php?accion=Mas&id_busqueda=_Y0wyY1ppR3hqa2xCZjhyNDREcndPME1hRHdnbXJaWkV4ME9mbW5XN0RIekkyMkpHdGErVHNZYURoUjViSlJua0JMQzltMS9nVXNUMjlFa1VBbG4yT2hoSGNEUGw5WjhoaGZtQzQrSHJGVTFqS2RUODJkL0RaMUxGSEpwemhPeU5DL1o3aW1NYmVpVG93S3d6UHAzWXNXS0c3VWlsQTBuRU9zL0FjQW1jSGRnamEzTFU2aERSUzVFZDBmS0NLNjljeTZOanZNWWxjNEVoZHZhc2pwU2hKZUJGTEg1T3ViREVLTGhlRXp4MjZXZHhjWkN2YmcxQ0NXNHNxNExJVmYrSGFaSDdoRU0xL0Q2OE9VR1R2c2ZSOEE9PQ,,-0-450"

# property_ids_url = "https://subastas.boe.es/reg/subastas_ava.php?accion=Mas&id_busqueda=_ZURMUzR4WmN2Uk1VS1dkeEo1ZmFId0ppTUxjVVVQclNub3BnckVtQzdxeDJEY2Z6V2dOWVJRT0pqTlFlS01YVUUzZFdISmJpeTBEQVN1TVpnSGpodzNTYkhxNWo3ejY4eGNQVGZ1dHhCQ1hXM0lFZG1tMEhETGtBSE13ZGtiREZOSUh1d3RXMWFIZkVqNCtGbUhtWm1nd0Q4QWNYYkJrZHpqdzVDVFNOY094MFZkdkF5U2kvSXQ4Z0N4STBZakNwV0hnUnByWG5mMXRTZStidnc3YlByaEMvZXFSVkdDZXlJazF0eDlTK0hETTdMT0h4S3p6NEJJd21hdHEreXpZZVhyclFsdFI4RTRNVlBJbGlGbThicTNpT1NTWGVaNVRvRlpnMGN3Ky8xeHJHQmJDaGxBR0g3Um12V2FsbEJaZnM,-0-50"

# # Fetch the HTML content
# response = requests.get(property_ids_url)
# html_content = response.text
# # Parse the HTML using BeautifulSoup
# soup = BeautifulSoup(html_content, "html.parser")

# # Extract property IDs from the <a> elements with class 'resultado-busqueda-link-otro'
# properties_list = [a['title'] for a in soup.find_all('a', class_='resultado-busqueda-link-otro')]

# # Initialize an empty DataFrame to store the property info
# df = pd.DataFrame()

# # Loop through property IDs and extract info
# for property_id_full in properties_list:
#     # Extract the relevant part of the property ID
#     property_id = property_id_full.split(" ")[1]
    
#     # Construct property URL
#     property_url = f"https://subastas.boe.es/detalleSubasta.php?idSub={property_id}"
    
#     # Fetch the property page
#     response = requests.get(property_url)
#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Check if the property page has the expected structure
#     table_div = soup.find("div", id="idBloqueDatos1")
#     if table_div:
#         # Extract info from the table
#         table = table_div.find("table")
#         rows = table.find_all("tr")
        
#         property_data = {}
#         for row in rows:
#             cols = row.find_all(["th", "td"])  # Including th (headers) and td (values)
#             header = cols[0].get_text(strip=True)
#             value = cols[1].get_text(strip=True) if len(cols) > 1 else ""  # Handle cases where there's no value cell
#             property_data[header] = value
        
#         # Append the property info to the DataFrame
#         df = pd.concat([df, pd.DataFrame(property_data, index=[0])], ignore_index=True)


In [3]:
# # Create a list to store dictionaries of property data
# data_list = []

# # Iterate through each property
# for property_info in properties_list:
#     if property_info.startswith('Subasta SUB-'):
#         property_id = property_info.split(' ')[-1]
        
#         url = f'https://subastas.boe.es/reg/detalleSubasta.php?idSub={property_id}&ver=3&idBus=_aGZaODZVK09aRlgzcFpxSzQ4SWlXa3ZOSDdnaFNaMkg1OWV0U2pHWjFwUEc2aHo5TVl3QjNtUnJUQ1FGS2ZoYmdoYS94cncxU1ZhYjNYMXFndzdyaVNvUlZpSUJHdm83eEVla3UyMElJYjJFcHpGdFB4N0RWQ20xRThPNFFrUStieGtWOWhaTlJuNGtqUXlyV3V3dFdYbWFPd3BGMnh3dlZaK2tXaElFQjVvZmhiUkdFMTJJYTYrUnRsMkU0enNHektmOUZRdnpYQmZHcERYRlY2bUdDVHE1UFZaLzJRS1BDMFlPNVhCb2R1SEYrOGljOW85djdqdE1zeWZySnNZQS8wTkhSS1lvWkxVMGJ5enJkWUhLdEhmYy9JY3g0YlF3YnpZdWo1aVhlOGs9--50&idLote=&numPagBus='

#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'html.parser')

#         div_id_bloque_datos3 = soup.find('div', id='idBloqueDatos3')
#         if div_id_bloque_datos3:
#             data_divs = div_id_bloque_datos3.find_all('div', class_='bloque')
#             for data_div in data_divs:
#                 property_data = {'Descripción': property_info, 'Property ID': property_id}
                
#                 property_table = data_div.find('table')
#                 if property_table:
#                     rows = property_table.find_all('tr')
#                     for row in rows:
#                         th = row.find('th')
#                         td = row.find('td')
#                         if th and td:
#                             property_data[th.get_text(strip=True)] = td.get_text(strip=True)
                
#                 data_list.append(property_data)

# # Create a DataFrame from the list of dictionaries
# bienes_df = pd.DataFrame(data_list)
# bienes_df.rename(columns={'Property ID':'Identificador'},inplace=True)
# columns_to_drop = ['Título jurídico', 'Información adicional', 'Valor Subasta', 'Valor de tasación','Importe del depósito', 'Puja mínima', 'Tramos entre pujas','IDUFIR']

# bienes_df = bienes_df.drop(columns=columns_to_drop)

# main_df = df.merge(bienes_df, on='Identificador', how='inner')
# main_df = main_df.dropna(subset=['Dirección', 'Código Postal'])
# main_df['Localidad'] = main_df['Localidad'].fillna('Consuela')
# main_df.fillna('No consta', inplace=True)
# columns_to_convert = ['Dirección', 'Código Postal', 'Localidad', 'Provincia']
# main_df[columns_to_convert] = main_df[columns_to_convert].apply(lambda x: x.str.upper())

In [4]:
# # Drop the 'Forma adjudicación' column
# df.drop('Forma adjudicación', axis=1, inplace=True)

# # Convert 'Fecha de inicio' and 'Fecha de conclusión' to datetime
# def parse_datetime(date_string):
#     return pd.to_datetime(date_string.split('CET')[0].strip(), format='%d-%m-%Y %H:%M:%S')

# df['Fecha de inicio'] = df['Fecha de inicio'].apply(parse_datetime)
# df['Fecha de conclusión'] = df['Fecha de conclusión'].apply(parse_datetime)

# df['Fecha de inicio'] = df['Fecha de inicio'].dt.date
# df['Fecha de conclusión'] = df['Fecha de conclusión'].dt.date

# # Convert numeric columns to float with correct formatting
# numeric_columns = ['Cantidad reclamada', 'Valor subasta', 'Tasación', 'Importe del depósito']
# for col in numeric_columns:
#     df[col] = df[col].apply(lambda x: x.replace(' €', '').replace('.', '').replace(',', '.') if isinstance(x, str) else x)
#     df[col] = pd.to_numeric(df[col], errors='coerce')

# special_numeric_columns = ['Puja mínima', 'Tramos entre pujas']

# for col in special_numeric_columns:
#     df[col] = df[col].apply(lambda x: 0 if x == 'Sin tramos' else x)
#     df[col] = df[col].apply(lambda x: x.replace(' €', '').replace('.', '').replace(',', '.') if isinstance(x, str) else x)
#     df[col] = pd.to_numeric(df[col], errors='coerce')
    
# # Fill NaN values
# columns_to_fill = ['Valor subasta', 'Puja mínima', 'Cantidad reclamada', 'Tasación','Tramos entre pujas','Importe del depósito']
# df[columns_to_fill] = df[columns_to_fill].fillna(0)
# df['Cuenta expediente'] = df['Cuenta expediente'].fillna('0123 4567 89 0987 65')

# # Replace 'Sin lotes' with 0
# df['Lotes'] = df['Lotes'].replace('Sin lotes', 0)

# # Convert the 'Lotes' column to integer type
# df['Lotes'] = df['Lotes'].astype(int)

In [5]:
# # List of first words to filter out
# words_to_filter = [
#     'LG', 'TN', 'PEBRES', 'VIVIENDA', 'CP', 'MN', '30005', 'PD', 'AR', 'CN',
#     'GREGORIO', 'NUEVA', 'POU', 'RAMON', 'PARAJE', 'PARTIDA', 'DISEMINADO',
#     'SAN', 'PEREIJO.', 'SANTA', 'CAÑOCLAR', 'NO', 'CLOSA', 'GRAN', 'LA', 'SUERTE'
# ]

# # Filter out rows with the specified first words in the address
# filtered_df = main_df[~main_df['Dirección'].str.split().str[0].isin(words_to_filter)]

# # Mapping of first words to standardized forms
# standardized_mapping = {
#     'CL': 'CALLE',
#     'LUGAR': 'LUGAR',
#     'CAMINO': 'CAMINO',
#     'CALLE': 'CALLE',
#     'AV.': 'AVENIDA',
#     'CL.': 'CALLE',
#     'C/': 'CALLE',
#     'C/COSTERETA,': 'CALLE COSTERETA',
#     'UR': 'URBANIZACION',
#     'C.': 'CALLE',
#     'CM': 'CAMINO',
#     'CR': 'CALLE',
#     'C/SANTIAGO': 'CALLE SANTIAGO',
#     'PA': 'PASAJE',
#     'CARRETERA': 'CARRETERA',
#     'AV': 'AVENIDA',
#     'PJ': 'PASAJE',
#     'C/VIRGEN': 'CALLE VIRGEN',
#     'CALLE:': 'CALLE',
#     'PARQUE': 'PARQUE',
#     'PZ': 'PLAZA',
#     'POLIGONO': 'POLIGONO',
#     'PG': 'POLIGONO',
#     'POLÍGONO': 'POLIGONO',
#     'PLAZA': 'PLAZA',
#     'C/BLASCO': 'CALLE BLASCO',
#     'PASAJE': 'PASAJE',
#     'C/CHILE': 'CALLE CHILE',
#     'C/TEIDE': 'CALLE TEIDE',
#     'C/ALFONSO': 'CALLE ALFONSO',
#     '"EDIFICIO': 'EDIFICIO',
#     'CL/': 'CALLE',
#     'AVENIDA': 'AVENIDA',
#     'NUMERO': 'NUMERO',
#     'URB.': 'URBANIZACION',
#     'C/ANTONIO': 'CALLE ANTONIO',
#     'URB': 'URBANIZACION',
#     'RONDA': 'RONDA',
#     'RAMBLA': 'RAMBLA',
#     'URBANIZACION': 'URBANIZACION',
#     'C/LUÍS': 'CALLE LUIS',
#     'C/NICOLÁS': 'CALLE NICOLAS',
#     'PG.': 'POLIGONO',
#     'AVDA': 'AVENIDA',
#     'TRAVESIA': 'TRAVESIA',
#     'RAMAL': 'RAMAL',
#     'CTRA': 'CARRETERA',
#     'PASEO': 'PASEO',
#     'SOLAR': 'SOLAR',
#     'CAMI': 'CAMINO',
#     'C/SANTO': 'CALLE SANTO',
#     'AVD': 'AVENIDA',
#     'PARCELA': 'PARCELA',
#     'AVDA.': 'AVENIDA'
# }

# # Apply standardization to the 'Dirección' column
# filtered_df['Dirección'] = filtered_df['Dirección'].apply(lambda address: ' '.join([standardized_mapping.get(address.split()[0], address.split()[0].upper())] + address.split()[1:]))

# # Function to extract the address part until the first number
# def extract_address(address):
#     match = re.match(r'^(.*?\d+)\b', address)
#     if match:
#         return match.group(1)
#     return address

# # Create the "Dirección Mapa" column
# filtered_df['Dirección Mapa'] = filtered_df['Dirección'].apply(extract_address)

In [ ]:
# property_bienes_url = 'https://subastas.boe.es/reg/detalleSubasta.php?idSub=SUB-AT-2023-23R3886001041&ver=3&idBus=_Y0wyY1ppR3hqa2xCZjhyNDREcndPME1hRHdnbXJaWkV4ME9mbW5XN0RIekkyMkpHdGErVHNZYURoUjViSlJua0JMQzltMS9nVXNUMjlFa1VBbG4yT2hoSGNEUGw5WjhoaGZtQzQrSHJGVTFqS2RUODJkL0RaMUxGSEpwemhPeU5DL1o3aW1NYmVpVG93S3d6UHAzWXNXS0c3VWlsQTBuRU9zL0FjQW1jSGRnamEzTFU2aERSUzVFZDBmS0NLNjljeTZOanZNWWxjNEVoZHZhc2pwU2hKZUJGTEg1T3ViREVLTGhlRXp4MjZXZHhjWkN2YmcxQ0NXNHNxNExJVmYrSGFaSDdoRU0xL0Q2OE9VR1R2c2ZSOEE9PQ,,-0-450&idLote=&numPagBus='

# property_bienes_url = "https://subastas.boe.es/reg/detalleSubasta.php?idSub=SUB-AT-2023-23R3886001041&ver=3&idBus=_aGZaODZVK09aRlgzcFpxSzQ4SWlXa3ZOSDdnaFNaMkg1OWV0U2pHWjFwUEc2aHo5TVl3QjNtUnJUQ1FGS2ZoYmdoYS94cncxU1ZhYjNYMXFndzdyaVNvUlZpSUJHdm83eEVla3UyMElJYjJFcHpGdFB4N0RWQ20xRThPNFFrUStieGtWOWhaTlJuNGtqUXlyV3V3dFdYbWFPd3BGMnh3dlZaK2tXaElFQjVvZmhiUkdFMTJJYTYrUnRsMkU0enNHektmOUZRdnpYQmZHcERYRlY2bUdDVHE1UFZaLzJRS1BDMFlPNVhCb2R1SEYrOGljOW85djdqdE1zeWZySnNZQS8wTkhSS1lvWkxVMGJ5enJkWUhLdEhmYy9JY3g0YlF3YnpZdWo1aVhlOGs9-0-50&idLote=&numPagBus="

# # Fetch the HTML content
# response = requests.get(property_bienes_url)
# html_content = response.text

# # Parse the HTML using BeautifulSoup
# soup = BeautifulSoup(html_content, "html.parser")